# Tekla Modeling Ahyeon API (기본편)

> 아현 푸르지오 모델링 진행 하면서 생산성을 향상 시킨 툴에 관한 설명서이다. 
> *(2021-04-02)*

> 기본편과 응용편으로 나누어진다. 
> 기본편은 여러 프로그램에 공통적으로 쓰인 함수를 위주로 설명하고
> 응용편은 각 부재별 위주로 설명한다.
> 추가적으로 REVIT API와 TEKLA API를 활용하여 REVIT과 TEKLA간의 골조모델 변환하는 방법도 작성 할 수 있길 희망한다.
> *(2021-04-06)*

# 1.기본 Setting
먼저 시작하기에 앞서 Visual Studio 기본 세팅을 해야한다. (Visual Stuidio와 Tekla는 설치되어 있다는 가정하에 진행한다.)

### 1.1 TEKLA 도구 설치

    1. 도구>NuGet 패키지 관리자> 솔루션용 NuGet 패키지 관리 클릭

<img src="TeklaNuget01.png">

    2. 찾아보기> Tekla입력> 해당하는 버전 Down 설치 클릭 

<img src="TeklaNuget02.png">

### 1.2 Using문을 활용한 기본 Tekla Class Import

    1. 자주 사용하는 Class들을 Import한 것으로 프로그래머에 의해 조정가능하다.

In [39]:
//기본 Class
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;

In [42]:
//테클라 코딩에 추가해야 할 부분
using Tekla.Structures.Model;
using Tekla.Structures.Geometry3d;
using Tekla.Structures.Model.UI;
using System.Diagnostics;
using Tekla.Structures.Model.Operations;
using System.Collections;
using Tekla.Structures.Datatype;
using Tekla.Structures.Solid;

-//오류 메세지 적게 보이게 하기 위해서..  

Unhandled Exception: (11,2): error CS1733: 식이 필요합니다.

# 2.주요 Reference 사이트
Tekla API를 이용하는데 있어 참고할 만한 사이트 2곳을 추천한다.

[테클라 API](https://developer.tekla.com/tekla-structures/api/14/8180).

[실제 사용 예시들 Github 참고](https://github.com/razorcx ).

# 3.주요 코드 설명
아현푸르지오 프로젝트를 진행하면서 자주 사용하는 함수들을 우선 정리하고, 그 함수들을 이용하여 만든 프로그램을 설명하겠다. 후자는 참고사항이며, Refactoring이 필요하다. 기본 개념은 Diagram으로 표현할까 고민중이다.

## 3.1 Part & RebarGroup 선택
Tekla 화면에서 Part & RebarGroup을 선택하는 기본 명령이다. 

In [45]:
//하나의 객체를 선택할 때


//Select Part
var pickedObjectPart = new Picker()
                    .PickObject(Picker.PickObjectEnum.PICK_ONE_PART, "Select Part");

var part = pickedObjectPart as Part;


//Select SingleRebar
var pickedObjectRebar = new Picker()
                    .PickObject(Picker.PickObjectEnum.PICK_ONE_REINFORCEMENT, "Select Part");

var part = pickedObjectRebar as SingleRebar;


//2개 이상의 객체를 선택할 때
//자료형이 Enumerator라 Currnet()와 MoveNext()가 있어야 한다.
//아래는 if while로 구현하였지만 foreach로 각각을 담는 방법도 있다고 한다. 이건 해봐야 한다.
//자세한 내용은 아래 두 사이트를 참고
//https://samsons.tistory.com/18
//https://docs.microsoft.com/ko-kr/dotnet/api/system.collections.generic.list-1.enumerator?view=net-5.0


//Select Multiple Parts
var pickedObjectParts = new Picker()
                .PickObjects(Picker.PickObjectsEnum.PICK_N_PARTS, "Select Parts");

if (pickedObjects.GetSize() < 1) return;

while (pickedObjects.MoveNext())
    {

        var rebar = pickedObjectParts.Current as Part;

        if (rebar == null)
            continue;
    }

//Select Multiple RebarGroup
var pickedObjectRebarGroup = new Picker()
                .PickObjects(Picker.PickObjectsEnum.PICK_N_REINFORCEMENTS, "Select RebarGroup");

if (pickedObjects.GetSize() < 1) return;

while (pickedObjects.MoveNext())
    {

        var rebar = pickedObjectRebarGroup.Current as RebarGroup;

        if (rebar == null)
            continue;


    }

-//오류 메세지 적게 보이게 하기 위해서..  

Unhandled Exception: (58,28): error CS1733: 식이 필요합니다.

> **위의 코드를 활용하여 RebarGroup의 FaterPart를 수정하는 예시.**

In [46]:
public void ChangeFaterPart()
        {
            var pickedObjectPart = new Picker()
                    .PickObject(Picker.PickObjectEnum.PICK_ONE_PART, "Select Fater Part");


            var pickedObjects = new Picker()
                .PickObjects(Picker.PickObjectsEnum.PICK_N_REINFORCEMENTS, "Select RebarGroup");

            if (pickedObjects.GetSize() < 1) return;

            while (pickedObjects.MoveNext())
            {

                var part = pickedObjectPart as Part;

                var rebar = pickedObjects.Current as RebarGroup;

                if (rebar == null)
                    continue;

                rebar.Father = part;

                
                rebar.Modify();


            }
            new Model().CommitChanges();
        }

-//오류 메세지 적게 보이게 하기 위해서..  

Unhandled Exception: (32,28): error CS1733: 식이 필요합니다.

## 3.2 Modify()와 Insert() 
Modify()와 Insert()는 유용하게 활용하면 Copy 기능을 활용하고 코드를 간결하게 만들 수 있는 도구이다.

In [50]:
//기둥에 Tie Bar를 모델링 할 경우 아래와 같이 Spacing을 TieBar의 갯수(strCount)만큼  
//아래와 같이 rebar.Insert()를 하면 객체를 Copy하면서 만들어 나갈 수 있다.

double spacing = 300;
double strCount = 3;

for (int i = 0; i < strCount; i++)
        {
            if (i == 0)
            {

                rebar.EndHook.Shape = RebarHookData.RebarHookShapeEnum.HOOK_90_DEGREES;
                rebar.EndHook.Shape = RebarHookData.RebarHookShapeEnum.CUSTOM_HOOK;
                rebar.EndHook.Angle = 90;
                rebar.EndHook.Length = 75.47;
                rebar.OnPlaneOffsets.Clear();
                rebar.OnPlaneOffsets.Add(spacing);

                rebar.Name = "TIE";

                rebar.Modify();
            }

            else
            {
                rebar.EndHook.Shape = RebarHookData.RebarHookShapeEnum.HOOK_90_DEGREES;
                rebar.EndHook.Shape = RebarHookData.RebarHookShapeEnum.CUSTOM_HOOK;
                rebar.EndHook.Angle = 90;
                rebar.EndHook.Length = 75.47;
                rebar.OnPlaneOffsets.Clear();
                rebar.OnPlaneOffsets.Add(spacing * (i+1));

                rebar.Insert(); //TieBar의 갯수가 늘어날 때마다 Copy의 기능을 같이 수행한다.

            }
        }

-//오류 메세지 적게 보이게 하기 위해서..  

Unhandled Exception: (37,28): error CS1733: 식이 필요합니다.

## 3.3 Create RebarGroup
RebarGroup을 만드는 방법이다.

In [52]:
using Tekla.Structures.Model;
using Tekla.Structures.Geometry3d;

public class Example
{
       public void Example1()
       {
           Beam Beam = new Beam(new Point(5000, 8000, 0), new Point(6000, 8000, 0));
           Beam.Profile.ProfileString = "250*250";
           Beam.Material.MaterialString = "K40-1";
           Beam.Finish = "PAINT";
           Beam.Insert();

           double MinimumX = Beam.GetSolid().MinimumPoint.X;
           double MinimumY = Beam.GetSolid().MinimumPoint.Y;
           double MinimumZ = Beam.GetSolid().MinimumPoint.Z;
           double MaximumX = Beam.GetSolid().MaximumPoint.X;
           double MaximumY = Beam.GetSolid().MaximumPoint.Y;
           double MaximumZ = Beam.GetSolid().MaximumPoint.Z;

           Polygon Polygon = new Polygon();
           Polygon.Points.Add(new Point(MinimumX, MaximumY, MinimumZ));
           Polygon.Points.Add(new Point(MinimumX, MinimumY, MinimumZ));
           Polygon.Points.Add(new Point(MinimumX, MinimumY, MaximumZ));
           Polygon.Points.Add(new Point(MinimumX, MaximumY, MaximumZ));

           Polygon Polygon2 = new Polygon();
           Polygon2.Points.Add(new Point(MaximumX, MaximumY, MinimumZ));
           Polygon2.Points.Add(new Point(MaximumX, MinimumY, MinimumZ));
           Polygon2.Points.Add(new Point(MaximumX, MinimumY, MaximumZ));
           Polygon2.Points.Add(new Point(MaximumX, MaximumY, MaximumZ));

           RebarGroup RebarGroup = new RebarGroup();
           RebarGroup.Polygons.Add(Polygon);
           RebarGroup.Polygons.Add(Polygon2);
           RebarGroup.RadiusValues.Add(40.0);
           RebarGroup.SpacingType = RebarGroup.RebarGroupSpacingTypeEnum.SPACING_TYPE_TARGET_SPACE;
           RebarGroup.Spacings.Add(30.0);
           RebarGroup.ExcludeType = RebarGroup.ExcludeTypeEnum.EXCLUDE_TYPE_BOTH;
           RebarGroup.Father = Beam;
           RebarGroup.Name = "RebarGroup";
           RebarGroup.Class = 3;
           RebarGroup.Size = "12";
           RebarGroup.NumberingSeries.StartNumber = 0;
           RebarGroup.NumberingSeries.Prefix = "Group";
           RebarGroup.Grade = "A500HW";
           RebarGroup.StartHook.Shape = RebarHookData.RebarHookShapeEnum.CUSTOM_HOOK;
           RebarGroup.StartHook.Angle = -90;
           RebarGroup.StartHook.Length = 3;
           RebarGroup.StartHook.Radius = 20;
           RebarGroup.EndHook.Shape = RebarHookData.RebarHookShapeEnum.CUSTOM_HOOK;
           RebarGroup.EndHook.Angle = -90;
           RebarGroup.EndHook.Length = 3;
           RebarGroup.EndHook.Radius = 20;
           RebarGroup.OnPlaneOffsets.Add(25.0);
           RebarGroup.OnPlaneOffsets.Add(10.0);
           RebarGroup.OnPlaneOffsets.Add(25.0);
           RebarGroup.StartPointOffsetType = Reinforcement.RebarOffsetTypeEnum.OFFSET_TYPE_COVER_THICKNESS;
           RebarGroup.StartPointOffsetValue = 20;
           RebarGroup.EndPointOffsetType = Reinforcement.RebarOffsetTypeEnum.OFFSET_TYPE_COVER_THICKNESS;
           RebarGroup.EndPointOffsetValue = 60;
           RebarGroup.FromPlaneOffset = 40;

           RebarGroup.Insert();

           RebarGroup.Name = "Modified Group 1";
           RebarGroup.Modify();
       }
}

-//오류 메세지 적게 보이게 하기 위해서..  

Unhandled Exception: (71,28): error CS1733: 식이 필요합니다.

## 3.4 속성을 추출하는 방법
Part & RebarGroup의 UserFields 속성을 추출하는 방법

In [53]:
public string UserFloorData(Part part)
        {
            var values = new Hashtable();
            part.GetAllUserProperties(ref values);

            // ifc story 스펠링 틀림 다른 프로젝트에서는 수정 필요
            var partFloor = values["IFC_BUILDING_STOREY"] as string;
            //var partFloor = values["IFC_BUILDING"] as string;
            return partFloor;
        }

        public string UserBuildingData(Part part)
        {
            var values = new Hashtable();
            part.GetAllUserProperties(ref values);

            // ifc story 스펠링 틀림 다른 프로젝트에서는 수정 필요
            var partFloor = values["IFC_BUILDING"] as string;
            return partFloor;
        }


        public string UserABC(Part part)
        {
            var values = new Hashtable();
            part.GetAllUserProperties(ref values);

            // ifc story 스펠링 틀림 다른 프로젝트에서는 수정 필요
            var partFloor = values["BIMQ_SECTION"] as string;
            //var partFloor = values["IFC_BUILDING"] as string;
            return partFloor;
        }

        //Part Property대로 자동 기입
        public void Run()
        {
            var pickedObjects = new Picker()
                    .PickObjects(Picker.PickObjectsEnum.PICK_N_PARTS);

            if (pickedObjects.GetSize() < 1) return;

            while (pickedObjects.MoveNext())
            {


                var part = pickedObjects.Current as Part;
                if (part == null)
                    continue;


                string buildingDataUser3 = UserABC(part);
                part.SetUserProperty("USER_FIELD_3", buildingDataUser3);

                
                string buildingData = UserBuildingData(part);
                string floorData = UserFloorData(part);

                part.SetUserProperty("USER_FIELD_1", buildingData);
                part.SetUserProperty("BIMQ_FLOOR", floorData);

                //Part에 속한 RebarGroup or 모델들 추출 나중에 delete 기존거 해야한다.
                ModelObjectEnumerator rebarModelHierarchic = part.GetChildren();
                foreach (var obj in rebarModelHierarchic)
                {
                    if (obj != null)
                    {
                        
                        RebarGroup rebarGroup = new RebarGroup();
                        rebarGroup = obj as RebarGroup;

                        if (rebarGroup != null)
                        {
                            rebarGroup.SetUserProperty("USER_FIELD_1", buildingData);
                            rebarGroup.SetUserProperty("USER_FIELD_2", floorData);

                            rebarGroup.SetUserProperty("BIMQ_FLOOR", floorData);

                            rebarGroup.SetUserProperty("BIMQ_SECTION", buildingDataUser3);
                        }
                    }
                }
                

                new Model().CommitChanges();




            }
            
-//오류 메세지 적게 보이게 하기 위해서..  

Unhandled Exception: (89,14): error CS1513: }가 필요합니다.

In [64]:
//전체 키값들 추출

public double UserFloorData(Part part)
        {
            var values = new Hashtable();
            part.GetAllUserProperties(ref values);


            Console.Write("3. 키값들만 출력\n\t");
            ICollection mykey = values.Keys;

            foreach (object obj in mykey)
                Console.Write("{0},", obj);
            return floorNumber;
        }
        
-//오류 메세지 적게 보이게 하기 위해서..  

Unhandled Exception: (17,28): error CS1733: 식이 필요합니다.

## 3.5 Array에서 최댓값,최솟값,세번째 값 추출하는 방법
Array의 내의 첫번째 값과 세번째 값을 추출할 수 있으면 Slab 배근 할 때 유용

In [ ]:
public ArrayList MinMaxValue(ArrayList distanceArray)
        {

            ArrayList arrayList = new ArrayList();
            int max = int.MinValue;
            int min = int.MaxValue;
            int maxIndex = 0;
            int minIndex = 0;
            

            for (int i = 0; i < distanceArray.Count; i++)
            {
                if (Convert.ToInt32(distanceArray[i]) > max)
                {
                    max = Convert.ToInt32(distanceArray[i]);
                    maxIndex = i;
                }
                if (Convert.ToDouble(distanceArray[i]) < min)
                {
                    min = Convert.ToInt32(distanceArray[i]);
                    minIndex = i;
                }

            }

            arrayList.Add(max);
            arrayList.Add(min);
            arrayList.Add(maxIndex);
            arrayList.Add(minIndex);

            return arrayList;

        }

-//오류 메세지 적게 보이게 하기 위해서..  

In [ ]:
public ArrayList FisrtThirdValue(ArrayList distanceArray)
        {

            ArrayList arrayList = new ArrayList();
            ArrayList sortList = new ArrayList(distanceArray);
            int max = int.MinValue;
            int third = int.MinValue;
            int maxIndex =1;
            int thirdIndex = 1;
            int countArray = distanceArray.Count;

            distanceArray.Sort();

            


                for (int i = 0; i < sortList.Count; i++)
            {
                if (Convert.ToInt32(distanceArray[countArray-1]) == Convert.ToInt32(sortList[i]))
                {
                    max = Convert.ToInt32(sortList[i]);
                    maxIndex = i;
                    
                }
                else if (Convert.ToInt32(distanceArray[countArray-3]) == Convert.ToInt32(sortList[i]))
                {
                    third = Convert.ToInt32(sortList[i]);
                    thirdIndex = i;
                    
                }

            }


           
            if (maxIndex != countArray - 1)
            {
                arrayList.Add(maxIndex);
                arrayList.Add(maxIndex + 1);
            }
            else
            {
                arrayList.Add(maxIndex);
                arrayList.Add(0);
            }

            if (thirdIndex != countArray - 1)
            {
                arrayList.Add(thirdIndex); 
                arrayList.Add(thirdIndex + 1);
            }
            else
            {
                arrayList.Add(thirdIndex);
                arrayList.Add(0);
            }


            
            
            return arrayList;

        }
        
-//오류 메세지 적게 보이게 하기 위해서..  

## 3.6 두 점을 이용하여 거리, 특정점을 추출하는 방법
두 점 사이의 거리를 이용하여 특정점 혹은 거리를 구하는 방법

In [ ]:
//거리 구하기

public double PointDistance(Point pt1, Point pt2)
        {
            double pointDistance = Math.Sqrt(Math.Pow(pt1.X - pt2.X, 2) + Math.Pow(pt1.Y - pt2.Y, 2) + Math.Pow(pt1.Z - pt2.Z, 2));

            return pointDistance;
        }
-//오류 메세지 적게 보이게 하기 위해서..  

In [ ]:
//1/3 지점 추출 parameterPercent에 0.3 입력하면 된다.

public Point RebarPointThreeFirst(Point startPoint, Point endPoint, double parameterPercent)
        {

            double threeFisrtX = 0;
            double threeFisrtY = 0;
            double threeFisrtZ = 0;

            if (startPoint.X < endPoint.X)
            {
                threeFisrtX = Math.Abs(startPoint.X - endPoint.X) * parameterPercent + startPoint.X;
            }
            else
            {
                threeFisrtX = Math.Abs(startPoint.X - endPoint.X) * (-1 * parameterPercent) + startPoint.X;
            }


            if (startPoint.Y < endPoint.Y)
            {
                threeFisrtY = Math.Abs(startPoint.Y - endPoint.Y) * parameterPercent + startPoint.Y;
            }
            else
            {
                threeFisrtY = Math.Abs(startPoint.Y - endPoint.Y) * (-1 * parameterPercent) + startPoint.Y;
            }


            if (startPoint.Z < endPoint.Z)
            {
                threeFisrtZ = Math.Abs(startPoint.Z - endPoint.Z) * parameterPercent + startPoint.Z;
            }
            else
            {
                threeFisrtZ = Math.Abs(startPoint.Z - endPoint.Z) * (-1 * parameterPercent) + startPoint.Z;
            }

            Point point = new Point(threeFisrtX, threeFisrtY, threeFisrtZ);

            return point;

        }
        
-//오류 메세지 적게 보이게 하기 위해서..  

In [ ]:
//1/3 지점 추출 parameterPercent에 0.3 입력하면 된다.


public Point RebarPointThreeFirstReverse(Point startPoint, Point endPoint, double parameterPercent)
        {

            double threeFisrtX = 0;
            double threeFisrtY = 0;
            double threeFisrtZ = 0;

            if (startPoint.X > endPoint.X)
            {
                threeFisrtX = Math.Abs(startPoint.X - endPoint.X) * parameterPercent + startPoint.X;
            }
            else
            {
                threeFisrtX = Math.Abs(startPoint.X - endPoint.X) * (-1 * parameterPercent) + startPoint.X;
            }


            if (startPoint.Y > endPoint.Y)
            {
                threeFisrtY = Math.Abs(startPoint.Y - endPoint.Y) * parameterPercent + startPoint.Y;
            }
            else
            {
                threeFisrtY = Math.Abs(startPoint.Y - endPoint.Y) * (-1 * parameterPercent) + startPoint.Y;
            }


            if (startPoint.Z > endPoint.Z)
            {
                threeFisrtZ = Math.Abs(startPoint.Z - endPoint.Z) * parameterPercent + startPoint.Z;
            }
            else
            {
                threeFisrtZ = Math.Abs(startPoint.Z - endPoint.Z) * (-1 * parameterPercent) + startPoint.Z;
            }

            Point point = new Point(threeFisrtX, threeFisrtY, threeFisrtZ);

            return point;

        }
        
-//오류 메세지 적게 보이게 하기 위해서..  

In [ ]:
Point thirtyPointFisrt = new Point();
                Point thirtyPointSecond = new Point();

                double distanceValue = PointDistance(firstPoint, secondPoint);
                double distanceValueSecond = PointDistance(secondPoint, thirdPoint);

                double percentValue = 0;
                double percentValueSecond = 0;

                //거리가 350이 되는 곳의 점을 찾아서 해당 점을 찾는 것이다.
                if (distanceValue <= 350)
                {
                    percentValue = (350 - distanceValue) / distanceValue;
                    thirtyPointFisrt = RebarPointThreeFirstReverse(firstPoint, secondPoint, percentValue);

                }
                else
                {
                    percentValue = 1 - (350 / distanceValue);
                    thirtyPointFisrt = RebarPointThreeFirst(firstPoint, secondPoint, percentValue);

                }
                // 2번째
                if (distanceValueSecond <= 350)
                {
                    percentValueSecond = (350 - distanceValueSecond) / distanceValueSecond;
                    thirtyPointSecond = RebarPointThreeFirstReverse(secondPoint, thirdPoint, percentValueSecond);

                }
                else
                {
                    percentValueSecond = (350 / distanceValueSecond);
                    thirtyPointSecond = RebarPointThreeFirst(secondPoint, thirdPoint, percentValueSecond);

                }

                polygon.Points[0] = thirtyPointFisrt;
                polygon.Points[1] = secondPoint;
                polygon.Points.Add(thirtyPointSecond);

                rebar.EndHook.Shape = RebarHookData.RebarHookShapeEnum.CUSTOM_HOOK;
                rebar.EndHook.Angle = 0;
                rebar.EndHook.Length = 0;
                rebar.EndHook.Radius = 0;
                rebar.EndPointOffsetValue = 0;


                rebar.Polygons[0] = polygon;
                rebar.Class = 1;

                rebar.Insert();
                
-//오류 메세지 적게 보이게 하기 위해서..  

## 3.7 Solid 객체의 각 점을 추출하는 방법(Vector 활용)
MxPt와 MnPt는 객체의 Offset 값이 적용되어 있을 때 부정확한 값을 반환한다. 따라서 Solid 객체를 이용하여 Point를 반환하는 것이 정확하다. 

In [ ]:
                Point beamMaxPoint = part.GetSolid().MaximumPoint as Point;
                Point beamMinPoint = part.GetSolid().MinimumPoint as Point;

                //Face 추출 Refactoring 필요
                //이 부분이 Solid에서 Vector를 이용해 Face를 추출하는 부분이다. Offset이 있는 부분에서는 이와 같은 방법을 사용해야 한다.

                List<Point> facePointTop = new List<Point>();
                List<Point> facePointBot = new List<Point>();

                Solid solid = part.GetSolid();
                FaceEnumerator faceEnumerator = solid.GetFaceEnumerator();

                Face faceTop = faceEnumerator.Current as Face;

                while (faceEnumerator.MoveNext())
                {
                    var face = faceEnumerator.Current as Face;

                    if (face.Normal.Z >= 0.990 && face.Normal.Z <= 1.010)
                    {
                        faceTop = face as Face;
                    }

                }

                faceEnumerator.Reset();
                faceEnumerator.MoveNext();

                LoopEnumerator loopEnumeratorTop = faceTop.GetLoopEnumerator();


                //TopPoint
                while (loopEnumeratorTop.MoveNext())
                {
                    var pointFaceTop = loopEnumeratorTop.Current as Loop;

                    VertexEnumerator vertexEnumeratorTop = pointFaceTop.GetVertexEnumerator();

                    while (vertexEnumeratorTop.MoveNext())
                    {
                        facePointTop.Add(vertexEnumeratorTop.Current);
                    }
                    vertexEnumeratorTop.Reset();
                    vertexEnumeratorTop.MoveNext();
                }
                loopEnumeratorTop.Reset();
                loopEnumeratorTop.MoveNext();

                foreach (Point i in facePointTop)
                {
                    Point pointBot = new Point(i.X, i.Y, beamMinPoint.Z);
                    facePointBot.Add(pointBot);
                }

                //Edge 추출 끝
                
-//오류 메세지 적게 보이게 하기 위해서..  

## 3.8 철근정착길이, 이음길이를 적용
D10, D13 등 각 철근마다 콘크리트의 Material과 정착 방법, 이음에 따라 나누어진다. 이를 if문으로 정리 한 것이다.

In [ ]:
//0도 정착

public double ZeroSettleRebar(string size, string partMaterial)
        {

            double zeroSettle = 0;

            if (size == "10")
            {
                zeroSettle = 300;
            }
            else if (size == "13" && partMaterial == "C24")
            {
                zeroSettle = 380;
            }
            else if (size == "13" && partMaterial == "C27")
            {
                zeroSettle = 360;
            }
            else if (size == "16" && partMaterial == "C24")
            {
                zeroSettle = 570;
            }
            else if (size == "16" && partMaterial == "C27")
            {
                zeroSettle = 530;
            }
            else if (size == "19" && partMaterial == "C24")
            {
                zeroSettle = 680;
            }
            else if (size == "19" && partMaterial == "C27")
            {
                zeroSettle = 640;
            }

            zeroSettle = zeroSettle * -1;

            return zeroSettle;

        }
        
-//오류 메세지 적게 보이게 하기 위해서..  

In [ ]:
//90도 정착

public double RightSettleRebar(string size, string partMaterial)
        {

            double rightSettle = 0;

            
            if (size == "10" && partMaterial == "C24")
            {
                rightSettle = 170;
            }
            else if (size == "10" && partMaterial == "C27")
            {
                rightSettle = 160;
            }
            else if (size == "13" && partMaterial == "C24")
            {
                rightSettle = 220;
            }
            else if (size == "13" && partMaterial == "C27")
            {
                rightSettle = 210;
            }
            else if (size == "16" && partMaterial == "C24")
            {
                rightSettle = 330;
            }
            else if (size == "16" && partMaterial == "C27")
            {
                rightSettle = 310;
            }
            else if (size == "19" && partMaterial == "C24")
            {
                rightSettle = 400;
            }
            else if (size == "19" && partMaterial == "C27")
            {
                rightSettle = 380;
            }

            rightSettle = rightSettle * -1;

            return rightSettle;

        }
        
-//오류 메세지 적게 보이게 하기 위해서..  

In [ ]:
//이음

public double ZeroConnectRebar(string size, string partMaterial)
        {

            double zeroConnect = 0;


            if (size == "10" && partMaterial == "C24")
            {
                zeroConnect = 370;
            }
            else if (size == "10" && partMaterial == "C27")
            {
                zeroConnect = 350;
            }
            else if (size == "13" && partMaterial == "C24")
            {
                zeroConnect = 490;
            }
            else if (size == "13" && partMaterial == "C27")
            {
                zeroConnect = 460;
            }
            else if (size == "16" && partMaterial == "C24")
            {
                zeroConnect = 730;
            }
            else if (size == "16" && partMaterial == "C27")
            {
                zeroConnect = 690;
            }
            else if (size == "19" && partMaterial == "C24")
            {
                zeroConnect = 880;
            }
            else if (size == "19" && partMaterial == "C27")
            {
                zeroConnect = 830;
            }

            zeroConnect = zeroConnect * -1;

            return zeroConnect;

        }
-//오류 메세지 적게 보이게 하기 위해서..  

## 3.9 엑셀에서 데이터를 가져오는 방법
Excel을 통해 데이터를 가져오는 방법이다. Nuget Package에서 LinqToExcel 다운 받아야 한다.

In [60]:
using System.Linq;
using LinqToExcel;

Unhandled Exception: (2,7): error CS0246: LinqToExcel' 형식 또는 네임스페이스 이름을 찾을 수 없습니다. using 지시문 또는 어셈블리 참조가 있는지 확인하세요.

In [61]:
    public class ConnexionExcel
    {
        public string _pathExcelFile;
        public ExcelQueryFactory _urlConnexion;
        public ConnexionExcel(string path)
        {
            this._pathExcelFile = path;
            this._urlConnexion = new ExcelQueryFactory(_pathExcelFile);
        }
        public string PathExcelFile
        {
            get
            {
                return _pathExcelFile;
            }
        }
        public ExcelQueryFactory UrlConnexion
        {
            get
            {
                return _urlConnexion;
            }
        }
    }
    
-//오류 메세지 적게 보이게 하기 위해서..  

Unhandled Exception: (26,28): error CS1733: 식이 필요합니다.

In [62]:
   //엑셀 Column 각 Name에 맞게 추출
   
   public class SlabData
    {
        public string NAME
        {
            get;
            set;
        }
        public string SLAB_BOT
        {
            get;
            set;
        }

        public string SLAB_TOP
        {
            get;
            set;
        }

        public string TYPE
        {
            get;
            set;
        }

        public string X1
        {
            get;
            set;
        }

        public string X2
        {
            get;
            set;
        }

        public string X3
        {
            get;
            set;
        }

        public string X4
        {
            get;
            set;
        }

        public string Y1
        {
            get;
            set;
        }

        public string Y2
        {
            get;
            set;
        }
        public string Y3
        {
            get;
            set;
        }


    }

In [ ]:
public List<string> ExcelToLinqData(Part part)
        {
            string pathToExcelFile = @"D:\Project\아현 푸르지오\BASEMENTSLABREBAR.xlsx";
            ConnexionExcel ConxObject = new ConnexionExcel(pathToExcelFile);

            List<string> RebarXYValue = new List<string>();

            double floorData = IFCFloorData(part);

            var query1 = from a in ConxObject.UrlConnexion.Worksheet<SlabData>()
                         where a.NAME.Contains("S")
                         select a;


            // index 0 = TOPShort, 1 = BOTShort, 2 = TOPLong, 3 = BOTLong

            foreach (var result in query1)
            {

                if (part.Finish == result.NAME && Convert.ToInt32(floorData) >= Convert.ToInt32(result.SLAB_BOT) && Convert.ToInt32(floorData) <= Convert.ToInt32(result.SLAB_TOP))
                {

                    if (result.TYPE == "A")
                    {
                        RebarXYValue.Add(result.X1);
                        RebarXYValue.Add(result.X3);
                        RebarXYValue.Add(result.Y1);
                        RebarXYValue.Add(result.Y3);

                    }
                    else if(result.TYPE == "C")
                    {
                        RebarXYValue.Add(result.X3);
                        RebarXYValue.Add(result.X1);
                        RebarXYValue.Add(result.Y3);
                        RebarXYValue.Add(result.Y1);
                        // TYPE에 따라 X1 X3 의 상부하부가 다르다.
                    }
                    else
                    {
                        RebarXYValue.Add("10/300");
                        RebarXYValue.Add("10/300");
                        RebarXYValue.Add("10/300");
                        RebarXYValue.Add("10/300");

                        //일람에 없으면 10/300으로 통일

                    }

                }
                
            }

            return RebarXYValue;

        }


## 3.10 해당하는 파트의 RebarGroup을 삭제하는 방법
각 파트에 속한 객체를 수정하기 어려울 때는 아예 Delete()하여 새로 그리는 것이 빠르다.

In [66]:
public void DELETE()
        {
            var pickedObjects = new Picker()
                    .PickObjects(Picker.PickObjectsEnum.PICK_N_PARTS);

            if (pickedObjects.GetSize() < 1) return;

            while (pickedObjects.MoveNext())
            {


                var part = pickedObjects.Current as Part;
                if (part == null)
                    continue;



                //Part에 속한 RebarGroup or 모델들 추출 나중에 delete 기존거 해야한다.
                ModelObjectEnumerator rebarModelHierarchic = part.GetChildren();
                foreach (var obj in rebarModelHierarchic)
                {
                    if (obj != null)
                    {
                        RebarGroup rebarGroup = new RebarGroup();
                        rebarGroup = obj as RebarGroup;

                        if (rebarGroup != null)
                        {
                            rebarGroup.Delete();
                        }
                    }
                }

                new Model().CommitChanges();




            }
        }
        
-//오류 메세지 적게 보이게 하기 위해서..  

Unhandled Exception: (42,28): error CS1733: 식이 필요합니다.

## 3.11 RebarGroup이 Part를 벗어날 때 Part 안으로 위치시키는 방법
슬라브나 보 등의 부재안에 RebarGroup을 그릴 때 값을 음수 혹은 양수로 주어야 하는 경우가 있다. 이를 if문으로 가려낼 수 있다.

In [ ]:
public RebarGroup RebarGroupCorrection(RebarGroup rebarGroup, Point MinMaxPoint)
        {

            double rebarGroupTOPPointZ = rebarGroup.GetSolid().MaximumPoint.Z;
            double rebarGroupBOTPointZ = rebarGroup.GetSolid().MinimumPoint.Z;
            double cover = Convert.ToDouble(rebarGroup.OnPlaneOffsets[0]);

            if (rebarGroup.Name.Contains("TOP"))
            {

                if (MinMaxPoint.Z < rebarGroupTOPPointZ)
                {
                    rebarGroup.OnPlaneOffsets.Clear();
                    rebarGroup.OnPlaneOffsets.Add(cover * -1);
                    rebarGroup.Modify();
                }


            }
            else if (rebarGroup.Name.Contains("BOT"))
            {
                if (MinMaxPoint.Z > rebarGroupBOTPointZ)
                {
                    rebarGroup.OnPlaneOffsets.Clear();
                    rebarGroup.OnPlaneOffsets.Add(cover * -1);
                    rebarGroup.Modify();
                }

            }


            return rebarGroup;


        }
        
-//오류 메세지 적게 보이게 하기 위해서..  

## 3.12 내부와 외부를 구분하는 방법
Pt를 기준으로, 지하외벽의 피복두께를 고려한 모델링을 할 때 유용하다.

In [68]:
                            Point rebarGroupPoint1 = RebarGroup.GetSolid().MaximumPoint as Point;
							Point rebarGroupPoint2 = RebarGroup.GetSolid().MinimumPoint as Point;
							Point rebarGroupCopyPoint1 = RebarGroupCopy.GetSolid().MaximumPoint as Point;
							Point rebarGroupCopyPoint2 = RebarGroupCopy.GetSolid().MinimumPoint as Point;


							Point rebarPoint1 = new Point((rebarGroupPoint1.X + rebarGroupPoint2.X) / 2, (rebarGroupPoint1.Y + rebarGroupPoint2.Y) / 2, (rebarGroupPoint1.Z + rebarGroupPoint2.Z) / 2);
							Point rebarPoint2 = new Point((rebarGroupCopyPoint1.X + rebarGroupCopyPoint2.X) / 2, (rebarGroupCopyPoint1.Y + rebarGroupCopyPoint2.Y) / 2, (rebarGroupCopyPoint1.Z + rebarGroupCopyPoint2.Z) / 2);
							Point centerPoint = new Point(49800, 19300, 29840);


							double pointDistance1 = Math.Pow(centerPoint.X - rebarPoint1.X, 2) + Math.Pow(centerPoint.Y - rebarPoint1.Y, 2);
							double pointDistance2 = Math.Pow(centerPoint.X - rebarPoint2.X, 2) + Math.Pow(centerPoint.Y - rebarPoint2.Y, 2);

							if (pointDistance1 < pointDistance2)
							{
								RebarGroup.OnPlaneOffsets.Clear();
								RebarGroup.OnPlaneOffsets.Add(Convert.ToDouble(profileSplit[1]) / 2 - 50);
								RebarGroup.Size = widthRebarIN;
								RebarGroup.Spacings.Clear();
								RebarGroup.Spacings.Add(spacingRebarDoubleIN);
								RebarGroup.EndPointOffsetValue = rebarLengthEndIN;

								RebarGroup.Modify();

								RebarGroupCopy.OnPlaneOffsets.Clear();
								RebarGroupCopy.OnPlaneOffsets.Add(Convert.ToDouble(profileSplit[1]) / 2 - 70);
								RebarGroupCopy.Size = widthRebarOUT;
								RebarGroupCopy.Spacings.Clear();
								RebarGroupCopy.Spacings.Add(spacingRebarDoubleOUT);
								RebarGroupCopy.EndPointOffsetValue = rebarLengthEndOUT;

								RebarGroupCopy.Modify();


                            }
                            
-//오류 메세지 적게 보이게 하기 위해서..  

Unhandled Exception: (38,28): error CS1733: 식이 필요합니다.

## 3.13 RebarGroup의 높이를 수정하는 방법(StartPt,EndPt, Polygon, Range)
지하 외벽에 Copy를 할 때 높이가 달라지는 부분을 각 Part의 높이에 맞게 수정해야 한다. 이때 이러한 방법을 사용하면 유용하다.

In [ ]:
        public void HeightModify()
        {

            var pickedObjectPart = new Picker()
                    .PickObject(Picker.PickObjectEnum.PICK_ONE_PART, "Select Fater Part");


            var pickedObjects = new Picker()
                .PickObjects(Picker.PickObjectsEnum.PICK_N_REINFORCEMENTS);
            if (pickedObjects.GetSize() < 1) return;

            while (pickedObjects.MoveNext())
            {

                var part = pickedObjectPart as Part;

                var rebar = pickedObjects.Current as RebarGroup;

                if (rebar == null)
                    continue;

                string profileHeight = part.Profile.ProfileString;
                profileHeight = profileHeight.Split('X')[0];

                rebar.Father = part;
                VERHeight(rebar, Convert.ToDouble(profileHeight));

                rebar.Modify();


            }
            new Model().CommitChanges();
        }

        public void HeightModifyHOR()
        {

            var pickedObjectPart = new Picker()
                    .PickObject(Picker.PickObjectEnum.PICK_ONE_PART, "Select Fater Part");


            var pickedObjects = new Picker()
                .PickObjects(Picker.PickObjectsEnum.PICK_N_REINFORCEMENTS);
            if (pickedObjects.GetSize() < 1) return;

            while (pickedObjects.MoveNext())
            {

                var part = pickedObjectPart as Part;

                var rebar = pickedObjects.Current as RebarGroup;

                if (rebar == null)
                    continue;

                string profileHeight = part.Profile.ProfileString;
                profileHeight = profileHeight.Split('X')[0];

                rebar.Father = part;
                HORHeight(rebar, Convert.ToDouble(profileHeight));

                rebar.Modify();


            }
            new Model().CommitChanges();
        }

        public void VERHeight(RebarGroup rebar, double a)
        {


            Polygon polygon = new Polygon();
            Point pointStart = new Point();

            polygon = rebar.Polygons[0] as Polygon;

            pointStart = polygon.Points[0] as Point;
            Point pointEnd = new Point(pointStart.X, pointStart.Y, pointStart.Z + a);



            polygon.Points[0] = pointStart;
            polygon.Points[1] = pointEnd;

            rebar.Polygons[0] = polygon;

            rebar.Modify();

        }

        public void HORHeight(RebarGroup rebar, double a)
        {

            Point pointStart = new Point();

            pointStart = rebar.StartPoint as Point;
            Point pointEnd = new Point(pointStart.X, pointStart.Y, pointStart.Z + a);

            rebar.EndPoint = pointEnd;

            rebar.Modify();

        }
        
-//오류 메세지 적게 보이게 하기 위해서..  

## 3.14 강종을 회사 기준에 맞게 설정 & Main, Sub 설정 적용 
SD500, SD600은 D값에 따라 나누어지고, MAIN, SUB BAR는 Radius 값에 따라 정해진다.

In [56]:
        public string RebarGrade(string size)
        {
            string rebarGrade = string.Empty;

            if (size == "10" || size == "13")
            {
                rebarGrade = "SD500";
            }
            else if (size == "16" || size == "19" || size == "22" || size == "25")
            {
                rebarGrade = "SD600";
            }

            return rebarGrade;

        }
        
-//오류 메세지 적게 보이게 하기 위해서..  

Unhandled Exception: (18,28): error CS1733: 식이 필요합니다.

In [57]:
public double BendingRadiusMain(string size)
        {
            double bendingRadius = 0;

            if (size == "10")
            {
                bendingRadius = 30;
            }
            else if (size == "13")
            {
                bendingRadius = 40;
            }
            else if (size == "16")
            {
                bendingRadius = 50;
            }
            else if (size == "19")
            {
                bendingRadius = 60;
            }

            return bendingRadius;
        }
        
-//오류 메세지 적게 보이게 하기 위해서..  

Unhandled Exception: (25,28): error CS1733: 식이 필요합니다.

In [58]:
public double BendingRadiusSub(string size)
        {
            double bendingRadius = 0;

            if (size == "10")
            {
                bendingRadius = 20;
            }
            else if (size == "13")
            {
                bendingRadius = 27.5;
            }
            else if (size == "16")
            {
                bendingRadius = 32.5;
            }
            else if (size == "19")
            {
                bendingRadius = 57.5;
            }

            return bendingRadius;
        }
        
-//오류 메세지 적게 보이게 하기 위해서..  

Unhandled Exception: (25,28): error CS1733: 식이 필요합니다.